In [21]:
import pandas as pd
import numpy as np
import datetime as dt

In [4]:
dados = pd.read_csv("dados_com_resultado_esperado.csv")
dados.drop("Unnamed: 0", axis=1, inplace=True)
dados.head()

,CodigoNotaFiscal,CodigoCentroSaida,CodigoReceita,Volume,RelacaoAguaCimento Emitida,TipoLancamento,DataEmissaoNotaFiscal,CodigoCliente,CodigoMotorista,Status,...,TempoPermanenciaObra,ConsumoTotal,Idade,TipoTraco,CodigoAglomerante,FCK,RelacaoAguaCimento Receita,Rompimento7Dias,Rompimento28Dias,ResultadoEsperado
0,22,51020,11051,7.0,4.065,C,2010-07-02 06:29:25.063,11671051,173051,N,...,2010-07-02 01:43:00,2478.0,28,N,3051.0,15.0,0.9,10.73,21.46,0.256254
1,501,51120,11051,8.0,1.011,C,2010-07-06 10:43:21.2,11575051,517051,N,...,2010-07-06,2832.0,28,N,3051.0,15.0,0.9,10.69,18.45,23.266913
2,1204,51120,11051,8.0,1.011,C,2010-07-09 08:53:39.83,11003051,28079,N,...,2010-07-09 01:00:00,2832.0,28,N,3051.0,15.0,0.9,11.65,19.90,23.266913
3,1713,51020,11051,6.0,1.011,C,2010-07-12 11:16:20.723,7166051,340051,N,...,2010-07-12 00:20:00,2124.0,28,N,3051.0,15.0,0.9,10.50,18.37,23.266913
4,2090,51020,11051,8.0,1.011,C,2010-07-14 08:01:53.28,12403051,395051,N,...,NaN,2832.0,28,N,3051.0,15.0,0.9,11.69,19.55,23.266913


In [9]:
dados["DataEmissaoNotaFiscal"] = pd.to_datetime(dados["DataEmissaoNotaFiscal"])

In [35]:
curvas_crescimento = pd.read_csv("HistoricoCurvasCrescimento.csv", index_col="CodigoAglomerante")
curvas_crescimento["DataAlteracao"] = pd.to_datetime(curvas_crescimento["DataAlteracao"])
curvas_crescimento.head()

,Ano,Mes,ParametroX,ParametroY,DataAlteracao
CodigoAglomerante,,,,,
3051,2010,6,2.4050,-0.282,2010-07-09 08:47:36.923
3051,2010,6,2.4050,-0.282,2010-07-09 08:49:22.373
3051,2010,7,2.4050,-0.282,2010-07-09 08:53:40.973
4051,2010,7,1.9276,-0.183,2010-07-09 08:56:44.683
5051,2010,7,2.5370,-0.322,2010-07-09 08:57:42.057


In [49]:
def EncontrarProjecaoNotaFiscal(nota):
    curvas_do_aglomerante = curvas_crescimento.loc[nota["CodigoAglomerante"]]
    
    mask_mes = curvas_do_aglomerante["Mes"] == nota["DataEmissaoNotaFiscal"].month
    mask_ano = curvas_do_aglomerante["Ano"] == nota["DataEmissaoNotaFiscal"].year
    
    curvas_mes_ano_corretos = curvas_do_aglomerante[mask_mes & mask_ano]
    
    mask_data_emissao_nota = curvas_mes_ano_corretos["DataAlteracao"] <= nota["DataEmissaoNotaFiscal"]
    curvas_antes_emissao_nota = curvas_mes_ano_corretos[mask_data_emissao_nota]
    curvas_antes_emissao_nota = curvas_antes_emissao_nota.sort_values("DataAlteracao")
    
    if (curvas_antes_emissao_nota.empty):
        return np.NaN;
    
    curva_crescimento_momento = curvas_antes_emissao_nota.iloc[len(curvas_antes_emissao_nota) - 1]
    crescimento = ((curva_crescimento_momento["ParametroY"] * np.log(nota["Rompimento7Dias"])) + curva_crescimento_momento["ParametroX"] - 1)
    resultado_projetado = (crescimento * nota["Rompimento7Dias"]) + nota["Rompimento7Dias"]
    
    return resultado_projetado;

In [50]:
dados["ResultadoProjetado"] = dados.apply(EncontrarProjecaoNotaFiscal, axis = "columns")

In [68]:
mask_tem_projecao = dados["ResultadoProjetado"].notna()

dados = dados[mask_tem_projecao]

In [69]:
def EncontraModuloDiferencaEntreRealProjetado(nota):
    diferenca = (nota["Rompimento28Dias"] - nota["ResultadoProjetado"])/nota["Rompimento28Dias"]
    return np.abs(diferenca);

In [70]:
dados["DiferencaRealProjetado"] = dados.apply(EncontraModuloDiferencaEntreRealProjetado, axis = "columns")

In [71]:
dados.head()

,CodigoNotaFiscal,CodigoCentroSaida,CodigoReceita,Volume,RelacaoAguaCimento Emitida,TipoLancamento,DataEmissaoNotaFiscal,CodigoCliente,CodigoMotorista,Status,...,Idade,TipoTraco,CodigoAglomerante,FCK,RelacaoAguaCimento Receita,Rompimento7Dias,Rompimento28Dias,ResultadoEsperado,ResultadoProjetado,DiferencaRealProjetado
3,1713,51020,11051,6.0,1.011,C,2010-07-12 11:16:20.723,7166051,340051,N,...,28,N,3051.0,15.0,0.9,10.50,18.37,23.266913,18.868387,0.027130
4,2090,51020,11051,8.0,1.011,C,2010-07-14 08:01:53.280,12403051,395051,N,...,28,N,3051.0,15.0,0.9,11.69,19.55,23.266913,20.723169,0.060009
5,2140,51120,11051,8.0,1.011,C,2010-07-14 10:20:23.430,12403051,33079,N,...,28,N,3051.0,15.0,0.9,10.16,16.73,23.266913,18.332992,0.095815
6,2173,51020,11051,8.0,1.011,C,2010-07-14 12:00:04.647,12403051,7060,N,...,28,N,3051.0,15.0,0.9,13.52,17.19,23.266913,23.522874,0.368405
7,2306,51120,11051,8.0,1.011,C,2010-07-15 06:47:20.053,11344051,66079,N,...,28,N,3051.0,15.0,0.9,7.71,13.52,23.266913,14.392958,0.064568


In [72]:
dados.to_csv("dados_com_projecao.csv")